#Comparison of video game sales and GDP per resident in the world

The topic of this paper is the impact of GDP on video game sales in certain parts of the world. Main
the idea is to monitor GDP growth or decline depending on the year and place of sale, and to conclude whether and how much it affects
GDP affects game sales. The main sales areas that will be studied are the North
America and Japan (there is also data for Europe, but Europe has too many countries to
could conclude something). The assumption is that GDP affects sales, which I will try to prove.
I will also conclude which are the most popular video game genres, platforms and publishers.

1. Collection of data sources

The data was collected through the following sources:

Video Game Sales: https://www.kaggle.com/datasets/gregorut/videogamesales?resource=download

GDP by selected country: https://data.oecd.org/gdp/gross-domestic-product-gdp.htm#indicator-chart

The first dataset was downloaded from the Kaggle site. There are exactly 16,598 games in this dataset.
The following information can be found for each game:
* Rank – the place of the game by the total number of sales. First place is the best-selling game
looking at total sales worldwide.
* Name – name of the game
* Platform – the game release platform
* Year – the year the game was released
* Genre – genre of the game
* Publisher – publishing house of the game
* NA_Sales – sales in North America (in millions)
* EU_Sales – sales in Europe (in millions)
* JP_Sales – sales in Japan (in millions)
* Other_sales – sales in other parts of the world (in millions)
* Global_sales – total sales in the world

Second dataset is available at data.oecd.org. On this page it is possible to select a range
the year in which I want to take over GDP. The oldest game from the first data set is from 1980, and the newest from
2010 (there are also more recent ones, but in too few numbers, so these games will not be taken into account). Cause of
for this reason I will choose the range of years from 1980 to 2010
The essential data from this set for analysis are:
* Location – abbreviation of the country
* Indicator – indication that it is GDP ("gross domestic product" in English, GDP)
* Measure – dollar per capita label
* Time - year
* Value – GDP value

## 2. Cleaning data

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# importing libraries
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
# reading first dataframe - vgsales.csv
df1 = pd.read_csv("/content/drive/MyDrive/vgsales.csv")

In [4]:
# reading second dataframe - DP_LIVE_01062022142735936.csv
df2 = pd.read_csv("/content/drive/MyDrive/DP_LIVE_01062022142735936.csv")

## Cleaning first dataframe

In [5]:
 # Checking how many games have values that are null
 df1.isnull().sum()

Rank              0
Name              0
Platform          0
Year            271
Genre             0
Publisher        58
NA_Sales          0
EU_Sales          0
JP_Sales          0
Other_Sales       0
Global_Sales      0
dtype: int64

271 video games don't have a year record, so I'll discard those records (actually I'll take all values ​​where there are years)

In [6]:
# Creating new dataframe
df11 = df1[df1['Year'].notna()]

In [7]:
# Checking null values again
df11.isnull().sum()

Rank             0
Name             0
Platform         0
Year             0
Genre            0
Publisher       36
NA_Sales         0
EU_Sales         0
JP_Sales         0
Other_Sales      0
Global_Sales     0
dtype: int64

There are 36 games left without a record of who is the publisher, but that is not the focus of this seminar, so I will keep those records

In [8]:
df11.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16327 entries, 0 to 16597
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Rank          16327 non-null  int64  
 1   Name          16327 non-null  object 
 2   Platform      16327 non-null  object 
 3   Year          16327 non-null  float64
 4   Genre         16327 non-null  object 
 5   Publisher     16291 non-null  object 
 6   NA_Sales      16327 non-null  float64
 7   EU_Sales      16327 non-null  float64
 8   JP_Sales      16327 non-null  float64
 9   Other_Sales   16327 non-null  float64
 10  Global_Sales  16327 non-null  float64
dtypes: float64(6), int64(1), object(4)
memory usage: 1.5+ MB


Taking games from 1980 to 2010

In [9]:
df11['Year'] = df11['Year'].astype(int, errors = 'raise')

<ipython-input-9-e984808a3293>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df11['Year'] = df11['Year'].astype(int, errors = 'raise')


In [10]:
first_dataframe = df11[ (df11["Year"] > 1979) & (df11["Year"] < 2011) ]

# Cleaning second dataframe

In [11]:
df2

,"LOCATION,""INDICATOR"",""SUBJECT"",""MEASURE"",""FREQUENCY"",""TIME"",""Value"",""Flag Codes"""
0,"AUS,""GDP"",""TOT"",""USD_CAP"",""A"",""1980"",10470.286..."
1,"AUS,""GDP"",""TOT"",""USD_CAP"",""A"",""1981"",11947.245..."
2,"AUS,""GDP"",""TOT"",""USD_CAP"",""A"",""1982"",11981.66345,"
3,"AUS,""GDP"",""TOT"",""USD_CAP"",""A"",""1983"",12767.30554,"
4,"AUS,""GDP"",""TOT"",""USD_CAP"",""A"",""1984"",13593.136..."
...,...
1805,"SEN,""GDP"",""TOT"",""USD_CAP"",""A"",""2011"",2728.699987,"
1806,"SEN,""GDP"",""TOT"",""USD_CAP"",""A"",""2012"",2798.104708,"
1807,"SEN,""GDP"",""TOT"",""USD_CAP"",""A"",""2013"",2797.721619,"
1808,"SEN,""GDP"",""TOT"",""USD_CAP"",""A"",""2014"",2881.298928,"


In [12]:
for col in df2.columns:
    print(col)

LOCATION,"INDICATOR","SUBJECT","MEASURE","FREQUENCY","TIME","Value","Flag Codes"


Column division by commas

In [13]:
df2[['Location', 'Indicator', 'Subject', 'Measure', 'Frequency', 'Time', 'Value', 'Flag Codes']] = df2['LOCATION,"INDICATOR","SUBJECT","MEASURE","FREQUENCY","TIME","Value","Flag Codes"'].str.split(',', expand=True)

Deleting frist column

In [14]:
df22 = df2.drop('LOCATION,"INDICATOR","SUBJECT","MEASURE","FREQUENCY","TIME","Value","Flag Codes"', axis=1)

It is necessary to remove the quotation marks from the Time field, so that it can join the datasets by that field

In [15]:
df22 = df22.replace('"', '', regex=True)

In [16]:
df22['Time'] = df22['Time'].astype(int, errors = 'raise')

In [17]:
df22['Value'] = df22['Value'].astype(float, errors = 'raise')

In [18]:
df22

,Location,Indicator,Subject,Measure,Frequency,Time,Value,Flag Codes
0,AUS,GDP,TOT,USD_CAP,A,1980,10470.286937,
1,AUS,GDP,TOT,USD_CAP,A,1981,11947.245259,
2,AUS,GDP,TOT,USD_CAP,A,1982,11981.663450,
3,AUS,GDP,TOT,USD_CAP,A,1983,12767.305540,
4,AUS,GDP,TOT,USD_CAP,A,1984,13593.136482,
...,...,...,...,...,...,...,...,...
1805,SEN,GDP,TOT,USD_CAP,A,2011,2728.699987,
1806,SEN,GDP,TOT,USD_CAP,A,2012,2798.104708,
1807,SEN,GDP,TOT,USD_CAP,A,2013,2797.721619,
1808,SEN,GDP,TOT,USD_CAP,A,2014,2881.298928,


In [19]:
df22.rename(columns = {'Time':'Year'}, inplace = True)

At the center of the research are North America (I will take the 3 largest countries; USA, Canada and Mexico) and Japan

In [20]:
df22 = df22[ (df22["Location"] == "MEX") | (df22["Location"] == "USA")  | (df22["Location"] == "CAN" ) | (df22["Location"] == "JPN")]

I leave the location, time and GDP value

In [21]:
second_dataframe = df22.drop(['Indicator', 'Subject', 'Measure', 'Frequency', 'Flag Codes'], axis = 1)

In [22]:
second_dataframeJPN = second_dataframe[ second_dataframe["Location"] == "JPN"]

In [23]:
second_dataframeCAN = second_dataframe[ second_dataframe["Location"] == "CAN"]

In [24]:
second_dataframeUSA = second_dataframe[ second_dataframe["Location"] == "USA"]

In [25]:
second_dataframeMEX = second_dataframe[ second_dataframe["Location"] == "MEX"]

# Merging all dataframes

Adding Canada's GDP

In [26]:
first_dataframe = first_dataframe.merge(second_dataframeCAN, on = "Year")

In [27]:
first_dataframe = first_dataframe.drop(['Location'], axis = 1)

In [28]:
first_dataframe.rename(columns = {'Value':'GDP_Japan'}, inplace = True)

In [29]:
first_dataframe

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,GDP_Japan
0,1,Wii Sports,Wii,2006,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74,38120.176792
1,7,New Super Mario Bros.,DS,2006,Platform,Nintendo,11.38,9.23,6.50,2.90,30.01,38120.176792
2,8,Wii Play,Wii,2006,Misc,Nintendo,14.03,9.20,2.93,2.85,29.02,38120.176792
3,21,Pokemon Diamond/Pokemon Pearl,DS,2006,Role-Playing,Nintendo,6.42,4.52,6.04,1.37,18.36,38120.176792
4,101,The Legend of Zelda: Twilight Princess,Wii,2006,Action,Nintendo,3.83,2.19,0.60,0.70,7.31,38120.176792
...,...,...,...,...,...,...,...,...,...,...,...,...
12436,4782,Pengo,2600,1983,Adventure,Atari,0.38,0.02,0.00,0.00,0.40,14034.005498
12437,5384,Frostbite,2600,1983,Action,Activision,0.32,0.02,0.00,0.00,0.34,14034.005498
12438,5561,Frogger II: Threeedeep!,2600,1983,Action,Parker Bros.,0.31,0.02,0.00,0.00,0.33,14034.005498
12439,6009,Dolphin,2600,1983,Action,Activision,0.27,0.02,0.00,0.00,0.29,14034.005498


Adding Japan's GDP

In [30]:
first_dataframe = first_dataframe.merge(second_dataframeJPN, on = "Year")

In [31]:
first_dataframe = first_dataframe.drop(['Location'], axis = 1)

In [32]:
first_dataframe.rename(columns = {'Value':'GDP_Canada'}, inplace = True)

Adding USA's GDP

In [33]:
first_dataframe = first_dataframe.merge(second_dataframeUSA, on = "Year")

In [34]:
first_dataframe = first_dataframe.drop(['Location'], axis = 1)

In [35]:
first_dataframe.rename(columns = {'Value':'GDP_USA'}, inplace = True)

Adding Mexico's GDP

In [36]:
first_dataframe = first_dataframe.merge(second_dataframeMEX, on = "Year")

In [37]:
first_dataframe = first_dataframe.drop(['Location'], axis = 1)

In [38]:
first_dataframe.rename(columns = {'Value':'GDP_Mexico'}, inplace = True)

In [39]:
Merged_dataframe = first_dataframe

In [40]:
Merged_dataframe

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,GDP_Japan,GDP_Canada,GDP_USA,GDP_Mexico
0,1,Wii Sports,Wii,2006,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74,38120.176792,33638.841255,46234.105710,13725.383603
1,7,New Super Mario Bros.,DS,2006,Platform,Nintendo,11.38,9.23,6.50,2.90,30.01,38120.176792,33638.841255,46234.105710,13725.383603
2,8,Wii Play,Wii,2006,Misc,Nintendo,14.03,9.20,2.93,2.85,29.02,38120.176792,33638.841255,46234.105710,13725.383603
3,21,Pokemon Diamond/Pokemon Pearl,DS,2006,Role-Playing,Nintendo,6.42,4.52,6.04,1.37,18.36,38120.176792,33638.841255,46234.105710,13725.383603
4,101,The Legend of Zelda: Twilight Princess,Wii,2006,Action,Nintendo,3.83,2.19,0.60,0.70,7.31,38120.176792,33638.841255,46234.105710,13725.383603
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12436,4782,Pengo,2600,1983,Adventure,Atari,0.38,0.02,0.00,0.00,0.40,14034.005498,12098.354701,15508.007835,5757.163420
12437,5384,Frostbite,2600,1983,Action,Activision,0.32,0.02,0.00,0.00,0.34,14034.005498,12098.354701,15508.007835,5757.163420
12438,5561,Frogger II: Threeedeep!,2600,1983,Action,Parker Bros.,0.31,0.02,0.00,0.00,0.33,14034.005498,12098.354701,15508.007835,5757.163420
12439,6009,Dolphin,2600,1983,Action,Activision,0.27,0.02,0.00,0.00,0.29,14034.005498,12098.354701,15508.007835,5757.163420


Visualisations


The amount of games under the dataset, the games are relatively well balanced

In [41]:
import plotly.express as px

In [42]:
sumByPlatform = Merged_dataframe.groupby(["Platform"], sort=True).count()
top10 = sumByPlatform.sort_values("Rank", ascending=False).head(10)
top10['Platform'] = top10.index  # Add a 'Platform' column
fig = px.pie(top10, values='Rank', names='Platform', title='Amount of games by platform')
fig.show()


1. Top 10 most sold games

In [56]:
maxbyPlatform = Merged_dataframe.groupby(["Name"], sort=False)['Global_Sales'].max()
sortedMax = maxbyPlatform.sort_values(ascending=False)
top10 = sortedMax.head(10)
top10 = top10.reset_index()  # Reset the index to have 'Name' as a column
fig = px.bar(top10, x='Global_Sales', y='Name', orientation='h', title='Top 10 games')
fig.update_xaxes(title='Sales in millions')
fig.update_yaxes(title='Name')
fig.show()

Most sold game is Wii Sports

2. Most popular platforms

In [57]:
sumPlatforms = Merged_dataframe.groupby(["Platform"], sort=False)['Global_Sales'].sum()
sortedSum = sumPlatforms.sort_values(ascending=False)
sorted7 = sortedSum.head(7)
sorted7 = sorted7.reset_index()  # Reset the index to have 'Platform' as a column
fig = px.bar(sorted7, x='Platform', y='Global_Sales', title='Platforms with most sales')
fig.update_xaxes(title='Platform')
fig.update_yaxes(title='Sales in millions')
fig.show()


The best selling games are Playstation 2 games with even more than 1200 million in total sales.

3. Most popular genres

In [58]:
sumaGenre = Merged_dataframe.groupby(["Genre"], sort=False)['Global_Sales'].sum()
sortedGenre = sumaGenre.sort_values(ascending=False)
sorted7 = sortedGenre.head(7)
sorted7 = sorted7.reset_index()  # Reset the index to have 'Genre' as a column
fig = px.bar(sorted7, y='Genre', x='Global_Sales', orientation='h', title='Genres with most sales')
fig.update_xaxes(title='Sales in millions')
fig.update_yaxes(title='Genres with most selling')
fig.show()

Action games are by far the best sellers, followed by sports games and platformers

4. Top 10 publishers in the world

In [59]:
sumPublisher = Merged_dataframe.groupby(["Publisher"], sort=False)['Global_Sales'].sum()
sortedPublisher = sumPublisher.sort_values(ascending=False)
sorted10 = sortedPublisher.head(10)
sorted10 = sorted10.reset_index()  # Reset the index to have 'Publisher' as a column
fig = px.bar(sorted10, y='Publisher', x='Global_Sales', orientation='h', title='Total sales of top 10 publishers')
fig.update_xaxes(title='Sales in millions')
fig.update_yaxes(title='Publisher')
fig.show()


Nintendo leads convincingly.

5. Comparison of sales in North America and Japan by year

In [64]:
salesNA = Merged_dataframe.groupby(["Year"], sort=True)['NA_Sales'].sum()
salesJAP = Merged_dataframe.groupby(["Year"], sort=True)['JP_Sales'].sum()

fig = px.line(x=salesNA.index, y=salesNA.values, labels={'x': 'Year', 'y': 'Sales'}, title='Sales in North America vs. Sales in Japan')
fig.add_scatter(x=salesJAP.index, y=salesJAP.values, mode='lines', line=dict(color='red'), name='Sales in Japan')
fig.add_scatter(x=salesNA.index, y=salesNA.values, mode='lines', line=dict(color='green'), name='Sales in North America')
fig.show()

Visible decline after 2007, especially in North America, in Japan there is no visible decline, but there is no increase either.

6.  Comparison of GDP of all analyzed countries

In [49]:
GDP_byYearJapan = Merged_dataframe.groupby(["Year"], sort = True)['GDP_Japan'].max()
GDP_byYearUSA = Merged_dataframe.groupby(["Year"], sort = True)['GDP_USA'].max()
GDP_byYearCanada = Merged_dataframe.groupby(["Year"], sort = True)['GDP_Canada'].max()
GDP_byYearMexico = Merged_dataframe.groupby(["Year"], sort = True)['GDP_Mexico'].max()


In [66]:
fig = px.line(title="Comparison of GDP")
fig.add_scatter(x=GDP_byYearJapan.index, y=GDP_byYearJapan.values, mode='lines', name="GDP Japan", line=dict(color='green'))
fig.add_scatter(x=GDP_byYearUSA.index, y=GDP_byYearUSA.values, mode='lines', name="GDP USA", line=dict(color='blue'))
fig.add_scatter(x=GDP_byYearCanada.index, y=GDP_byYearCanada.values, mode='lines', name="GDP Canada", line=dict(color='red'))
fig.add_scatter(x=GDP_byYearMexico.index, y=GDP_byYearMexico.values, mode='lines', name="GDP Mexico", line=dict(color='orange'))
fig.update_xaxes(title="Year")
fig.update_yaxes(title="GDP")
fig.show()


Also a visible decline after ~ 2007. Assumption - World financial crisis after 2007

7. How much did the crisis affect sales in Europe?

In [51]:
salesEU = Merged_dataframe.groupby(["Year"], sort = True)['EU_Sales'].sum()

In [67]:
fig = px.line(x=salesEU.index, y=salesEU.values, title="Sales in Europe")
fig.update_xaxes(title="Year")
fig.update_yaxes(title="Sales in millions")
fig.update_layout(
    xaxis=dict(showgrid=True, gridwidth=0.4),
    yaxis=dict(showgrid=True, gridwidth=0.4)
)
fig.show()

A slight decline is visible after 2009


8. Comparison of Europe with North America and Japan

In [69]:
import plotly.graph_objects as go
fig = go.Figure()

fig.add_trace(go.Scatter(x=salesNA.index, y=salesNA.values, mode='lines', name="Sales in North America", line=dict(color='green')))
fig.add_trace(go.Scatter(x=salesJAP.index, y=salesJAP.values, mode='lines', name="Sales in Japan", line=dict(color='red')))
fig.add_trace(go.Scatter(x=salesEU.index, y=salesEU.values, mode='lines', name="Sales in Europe", line=dict(color='blue', dash='dot')))

fig.update_xaxes(title="Year")
fig.update_yaxes(title="Sales")
fig.update_layout(legend=dict(x=0, y=1), xaxis=dict(showgrid=True, gridwidth=0.4), yaxis=dict(showgrid=True, gridwidth=0.4))
fig.update_layout(title="Sales Comparison")
fig.show()


By far the most games were sold in North America, almost twice as much in Europe and the least in Japan

9. How does the rest of the world compare to the 3 main areas?

In [54]:
salesRest = Merged_dataframe.groupby(["Year"], sort = True)['Other_Sales'].sum()

In [71]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=salesNA.index, y=salesNA.values, mode='lines', name="Sales in North America", line=dict(color='green')))
fig.add_trace(go.Scatter(x=salesJAP.index, y=salesJAP.values, mode='lines', name="Sales in Japan", line=dict(color='red')))
fig.add_trace(go.Scatter(x=salesEU.index, y=salesEU.values, mode='lines', name="Sales in Europe", line=dict(color='orange')))
fig.add_trace(go.Scatter(x=salesRest.index, y=salesRest.values, mode='lines', name="Sales in rest of the world", line=dict(color='blue', dash='dot')))

fig.update_xaxes(title="Year")
fig.update_yaxes(title="Sales")
fig.update_layout(legend=dict(x=0, y=1), xaxis=dict(showgrid=True, gridwidth=0.4), yaxis=dict(showgrid=True, gridwidth=0.4))
fig.update_layout(title="Sales Comparison")
fig.show()

Until 1995 it was almost negligible, after 2005 it was even higher than in Japan